# 人狼BBSの発言から人狼推測モデルを学習してみる
## BBSログにdoc2vecを試す
ログを取得したのでそれらに対してやってみる

In [1]:
import MeCab
import pandas
import gensim

#### パース用の関数
MeCabを使って一つの対戦ログを一文ずつ形態素に分け，docに追加して返す

In [19]:
def parse(df_days):
    mecab = MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
    mecab.parse('')
    doc = []
    for text in df_days["content"]:
        sentence = []
        tmp_text = str(text)
        node = mecab.parseToNode(tmp_text)
        while node:
            nodef = node.feature.split(',')
            sentence.append(node.surface)
            node = node.next
        doc.append(sentence)
    return doc

#### doc2vecで使用できるようにする
lineにwords=['単語A','単語B', ...]のように入っていて，[i]がidとなる

In [3]:
def read_corpus(doc):
    for i, line in enumerate(doc):
        yield gensim.models.doc2vec.TaggedDocument(line, [i])

#### doc2vecによる学習
データを突っ込んでいく

In [20]:
dirs = "./bbsdata/"
df_vills = pandas.read_csv(dirs + str(1) + ".csv",index_col=0)
for i in range(2,100):
    df_vill = pandas.read_csv(dirs + str(i) + ".csv",index_col=0)
    df_vills = df_vills.append(df_vill)
doc = parse(df_vills)
train_corpus = list(read_corpus(doc))

In [21]:
model = gensim.models.doc2vec.Doc2Vec(size=400, min_count=10, iter=55)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)
model.save("test100model.model")

### 類似単語を調べる

In [27]:
model.most_similar("リーザ")

[('リザ', 0.8302386999130249),
 ('クララ', 0.7315598130226135),
 ('リナ', 0.7189242839813232),
 ('リズ', 0.7124338150024414),
 ('リデル', 0.70890212059021),
 ('アルビン', 0.695146918296814),
 ('エルナ', 0.6939751505851746),
 ('パメラ', 0.6938230991363525),
 ('パメ', 0.6925375461578369),
 ('カタリナ', 0.6751901507377625)]

### 類似文を調べる

In [33]:
model.docvecs.most_similar([458])

[(49283, 0.3972127437591553),
 (100519, 0.3952615261077881),
 (91717, 0.3946514129638672),
 (54051, 0.3885599970817566),
 (12758, 0.3876497745513916),
 (109888, 0.38557785749435425),
 (103625, 0.38408008217811584),
 (93119, 0.38372841477394104),
 (18747, 0.38095909357070923),
 (55014, 0.3807477653026581)]

In [31]:
print(''.join(train_corpus[458][0]))
print('\n')
print(''.join(train_corpus[95885][0]))

昨夜の乗り遅れ感でログ読んでも屋がいう商単品の白っぽさがよくわからなかったり。なんか●商希望から年や農の言動とか探っていこうかと思ってたけどどうもよくわからない。妙に対する反応は者>>506で屋に対するレスポンスが絶妙。今日の妙を見て年はどう感じたかちょっと気になってきた。と思ったら>>515>>516確認中。


人│兵羊農屋老年娘│者旅神商修　　今のところ●│農農＿農娘娘屋│＿＿＿＿＿　　こんな感じか○│屋屋＿老農＿老│＿＿＿＿＿▼│年神神年神神年│＿＿＿＿＿▽│神年＿娘年＿神│＿＿＿＿＿
